In [23]:
import os
import pickle

In [24]:
law_books={}
for filename in [f for f in os.listdir('./lawbook_data') if os.path.isfile(os.path.join('./lawbook_data', f))]:
    with open(os.path.join('./lawbook_data', filename),'rb') as f:
        law_books[filename.split('.')[0]] = pickle.load(f)

In [25]:
with open('example_court_rouling.txt') as f:
    example_text = f.read()

print(example_text[:25])

Urteil vom Landgericht Pa


In [26]:
import openai

In [ ]:
client = openai.OpenAI(api_key='')

response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to answer precisely"},
    {"role": "user", "content":
    f"""
You will be given a German Legal text and your task is to extract the references to law books from it (eg '§ 287 Abs. 2 ZPO'or similar).

You should extract the lawbook referenced, the paragraph referenced and the subparagraph referenced. Also return the original text how it's referenced.

The returned format should be in JSON.
Example for '§ 287 Abs. 2 Satz. 4 ZPO' and 'BGB § 315':
#############
{{
    "references":[
        {{
            "original":"§ 287 Abs. 2 ZPO",
            "lawbook": "ZPO",
            "paragraph": "287",
            "subparagraph": "2"
        }},
        {{
            "original":"BGB § 315",
            "lawbook": "BGB",
            "paragraph": "315",
            "subparagraph": ""
        }}
    ]
}}
You should only return this JSON and nothing else.

If multiple paragraphs are referenced (eg. '§§ 157, 133 BGB') you return them as separate entries.

You find and extract every single reference.

Law TEXT
################
{example_text}
################

    """}
  ],
temperature = 0.2
)

In [28]:
print(response.choices[0].message.content)

{
    "references":[
        {
            "original":"§ 540 Abs. 1 Nr. 1 ZPO",
            "lawbook": "ZPO",
            "paragraph": "540",
            "subparagraph": "1"
        },
        {
            "original":"§§ 540 Abs. 2, 313a Abs. 1 S. 1 ZPO",
            "lawbook": "ZPO",
            "paragraph": "540",
            "subparagraph": "2"
        },
        {
            "original":"§ 313a Abs. 1 S. 1 ZPO",
            "lawbook": "ZPO",
            "paragraph": "313a",
            "subparagraph": "1"
        },
        {
            "original":"§ 102 EnWG",
            "lawbook": "EnWG",
            "paragraph": "102",
            "subparagraph": ""
        },
        {
            "original":"§ 102 Abs.1 Satz 1 EnWG",
            "lawbook": "EnWG",
            "paragraph": "102",
            "subparagraph": "1"
        },
        {
            "original":"§ 102 Abs. 1 Satz 2 EnWG",
            "lawbook": "EnWG",
            "paragraph": "102",
            "subparagraph": "1"

In [29]:
response_json = response.choices[0].message.content
response_json

'{\n    "references":[\n        {\n            "original":"§ 540 Abs. 1 Nr. 1 ZPO",\n            "lawbook": "ZPO",\n            "paragraph": "540",\n            "subparagraph": "1"\n        },\n        {\n            "original":"§§ 540 Abs. 2, 313a Abs. 1 S. 1 ZPO",\n            "lawbook": "ZPO",\n            "paragraph": "540",\n            "subparagraph": "2"\n        },\n        {\n            "original":"§ 313a Abs. 1 S. 1 ZPO",\n            "lawbook": "ZPO",\n            "paragraph": "313a",\n            "subparagraph": "1"\n        },\n        {\n            "original":"§ 102 EnWG",\n            "lawbook": "EnWG",\n            "paragraph": "102",\n            "subparagraph": ""\n        },\n        {\n            "original":"§ 102 Abs.1 Satz 1 EnWG",\n            "lawbook": "EnWG",\n            "paragraph": "102",\n            "subparagraph": "1"\n        },\n        {\n            "original":"§ 102 Abs. 1 Satz 2 EnWG",\n            "lawbook": "EnWG",\n            "paragraph": "1

In [30]:
import json

In [31]:
references = json.loads(response_json)['references']
references

[{'original': '§ 540 Abs. 1 Nr. 1 ZPO',
  'lawbook': 'ZPO',
  'paragraph': '540',
  'subparagraph': '1'},
 {'original': '§§ 540 Abs. 2, 313a Abs. 1 S. 1 ZPO',
  'lawbook': 'ZPO',
  'paragraph': '540',
  'subparagraph': '2'},
 {'original': '§ 313a Abs. 1 S. 1 ZPO',
  'lawbook': 'ZPO',
  'paragraph': '313a',
  'subparagraph': '1'},
 {'original': '§ 102 EnWG',
  'lawbook': 'EnWG',
  'paragraph': '102',
  'subparagraph': ''},
 {'original': '§ 102 Abs.1 Satz 1 EnWG',
  'lawbook': 'EnWG',
  'paragraph': '102',
  'subparagraph': '1'},
 {'original': '§ 102 Abs. 1 Satz 2 EnWG',
  'lawbook': 'EnWG',
  'paragraph': '102',
  'subparagraph': '1'},
 {'original': '§ 36 Abs. 1 EnWG',
  'lawbook': 'EnWG',
  'paragraph': '36',
  'subparagraph': '1'},
 {'original': '§ 315 BGB',
  'lawbook': 'BGB',
  'paragraph': '315',
  'subparagraph': ''},
 {'original': '§ 433 Abs. 2 BGB',
  'lawbook': 'BGB',
  'paragraph': '433',
  'subparagraph': '2'},
 {'original': '§§ 133, 157 BGB',
  'lawbook': 'BGB',
  'paragraph

In [38]:
for reference in references:
    print(f"{reference['original']}-",end="")
    if reference['lawbook'] not in law_books.keys():
        print('Lawbook not found')
    elif reference['paragraph'] not in law_books[reference['lawbook']].paragraphs.keys():
        print('Paragraph not found')
    elif reference['subparagraph'] not in law_books[reference['lawbook']].paragraphs[reference['paragraph']].section_dict.keys():
        print('Subaragraph not found')
    else:
        print('ok')
    

§ 540 Abs. 1 Nr. 1 ZPO-ok
§§ 540 Abs. 2, 313a Abs. 1 S. 1 ZPO-ok
§ 313a Abs. 1 S. 1 ZPO-ok
§ 102 EnWG-Subaragraph not found
§ 102 Abs.1 Satz 1 EnWG-ok
§ 102 Abs. 1 Satz 2 EnWG-ok
§ 36 Abs. 1 EnWG-ok
§ 315 BGB-Subaragraph not found
§ 433 Abs. 2 BGB-ok
§§ 133, 157 BGB-Subaragraph not found
§ 157 BGB-Subaragraph not found
§ 87 Abs. 2 GWB-Subaragraph not found
§ 87 Abs. 1 Satz 1 GWB-Subaragraph not found
§ 87 Abs. 1 Satz 2 GWB-Subaragraph not found
§ 538 Abs. 1 ZPO-ok
§ 195 BGB-Subaragraph not found
§ 199 Abs. 1 BGB-ok
§ 204 Abs. 1 Nr. 3 BGB-ok
§ 690 Abs. 1 Nr. 3 ZPO-ok
§ 696 Abs. 3 ZPO-ok
§ 204 Abs. 2 BGB-ok
§ 148 ZPO-Subaragraph not found
§ 204 Abs. 2 S. 1 u. 3 BGB-ok
§ 204 Abs. 2 S. 4 BGB-ok
§ 250 ZPO-Subaragraph not found
§ 150 ZPO-Subaragraph not found
§ 287 Abs. 2 ZPO-ok
§ 287 Abs. 1 S. 1 ZPO-ok
§ 291 BGB-Subaragraph not found
§ 288 Abs. 1 BGB-ok
§ 696 Abs. 3 ZPO-ok
§ 91 Abs. 1 ZPO-ok
§ 708 Nr. 10 ZPO-Subaragraph not found
§ 713 ZPO-Subaragraph not found
§ 543 Abs. 2 ZPO-ok
